# Debugging Langchain
## Introduction

Abstractions are great, but sometimes we want to see what goes on under the hood of langchain. 
Here's a few options:

- https://python.langchain.com/docs/guides/debugging
- https://python.langchain.com/docs/modules/chains/how_to/debugging

## Installation

In [ ]:
%pip install -q langchain langchain-openai

Note: you may need to restart the kernel to use updated packages.


## Global debugging flag
First option is to enable global debugging in Langchain.

You can see , it shows a lot of the calls/events that go on internally.

In [3]:
import langchain
from langchain_openai import ChatOpenAI

# Enable global debugging
langchain.debug = True

llm = ChatOpenAI(model="gpt-4o-mini",temperature=0)
llm.invoke("Where is Auburn?")
# Disable global debugging
langchain.debug = False

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Where is Auburn?"
  ]
}
[llm/end] [llm:ChatOpenAI] [1.61s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Auburn can refer to several places, but the most notable ones are:\n\n1. **Auburn, Alabama**: A city in the southeastern United States, known for Auburn University, a major public research university.\n\n2. **Auburn, California**: A city located in Placer County, known for its historic Gold Rush-era downtown.\n\n3. **Auburn, Washington**: A city in King County, part of the Seattle metropolitan area.\n\nIf you have a specific Auburn in mind or need information about a different location, please let me know!",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
       

## Increase verbosity
Second one is to tell providers or objects to be more verbose

In [4]:
# Set verbose flag on objects
llm = ChatOpenAI(model="gpt-4o-mini",verbose=True, temperature=0)
result = llm.invoke("Where is Auburn?")
print(result.content)

Auburn can refer to several places, but the most notable ones are:

1. **Auburn, Alabama**: A city in the southeastern United States, known for being home to Auburn University.
2. **Auburn, California**: A city in Placer County, located in the Sierra Nevada foothills, known for its Gold Rush history.
3. **Auburn, Washington**: A city located in King County, part of the Seattle metropolitan area.

If you have a specific Auburn in mind, please provide more context!


Now without the verbose flag. As you can see , there is a not a lot of difference.

In [5]:
llm = ChatOpenAI(model="gpt-4o-mini",verbose=False, temperature=0)
result = llm.invoke("Where is Ghent?")
print(result.content)

Ghent is a city located in Belgium, situated in the Flanders region. It is the capital of the East Flanders province and is known for its rich history, medieval architecture, and vibrant cultural scene. Ghent lies at the confluence of the Rivers Scheldt and Lys, making it an important historical trade center. The city is also home to several notable landmarks, including the Gravensteen castle and St. Bavo's Cathedral, which houses the famous Ghent Altarpiece.


## Using callbacks
A third useful way is to use callbacks to listen into objects events. We created our own helper in these lessons to listen to events.

In [7]:
from langchain_core.tracers.stdout import ConsoleCallbackHandler

# A console callback handler
console_callback = ConsoleCallbackHandler()

# Added to the model as callback option
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0 , callbacks=[console_callback])

result = llm.invoke("Where is Auburn?")
print(result.content)

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Where is Auburn?"
  ]
}
[llm/end] [llm:ChatOpenAI] [2.89s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Auburn can refer to several places, but the most notable ones are:\n\n1. **Auburn, Alabama**: A city in the southeastern United States, known for being home to Auburn University.\n2. **Auburn, California**: A city in Placer County, located in the Sierra Nevada foothills, known for its Gold Rush history.\n3. **Auburn, Washington**: A city located in King County, part of the Seattle metropolitan area.\n\nIf you have a specific Auburn in mind, please provide more context!",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",

## Callback for LCEL syntax
We also use the callback handler with the LCEL syntax

LangChain Expression Language (LCEL)

https://python.langchain.com/v0.1/docs/expression_language/

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.callbacks.tracers import ConsoleCallbackHandler

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
output_parser = StrOutputParser()

#LCEL chain
chain = prompt | model | output_parser

# Adding the callback
callbacks = [ConsoleCallbackHandler()]
chain.invoke({"topic": "ice cream"}, config={'callbacks': callbacks})


[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "topic": "ice cream"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "topic": "ice cream"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: tell me a joke about ice cream"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Why did the ice cream truck break down?\n\nBecause it had a rocky road!",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
         

'Why did the ice cream truck break down?\n\nBecause it had a rocky road!'